In [1]:
import numpy as np
import sympy as sym
import numba
import pydae.build as db

\begin{eqnarray}
\dot \delta &=& \Omega_b \left(\omega - \omega_s\right)\\
\dot \omega &=& 1/(2 H) \left(p_m - p_e - D (\omega - \omega_s) \right)\\
\end{eqnarray}

$$ \sf
\Omega_{b} \left(\sf \omega - \omega_{s}\right)
$$

## System definition 

In [2]:
params_dict = {'X_d':1.81,'X1d':0.3,'T1d0':8.0,
               'X_q':1.76,'X1q':0.65,'T1q0':1.0,
               'R_a':0.003,'X_line': 0.05, 
               'H':3.5,'D':1.0,
               'Omega_b':2*np.pi*50,'omega_s':1.0,
               'v_0':1.0,'theta_0':0.0}


u_ini_dict = {'P_t':0.8, 'Q_t':0.2}  # for the initialization problem
u_run_dict = {'p_m':0.8,'v_f':1.0}  # for the running problem (here initialization and running problem are the same)


x_list = ['delta','omega','e1q','e1d']    # [inductor current, PI integrator]
y_ini_list = ['i_d','i_q','v_1','theta_1','p_m','v_f'] # for the initialization problem
y_run_list = ['i_d','i_q','v_1','theta_1','P_t','Q_t'] # for the running problem (here initialization and running problem are the same)

sys_vars = {'params_dict':params_dict,
            'u_list':u_run_dict,
            'x_list':x_list,
            'y_list':y_run_list}

exec(db.sym_gen_str())  # exec to generate the required symbolic varables and constants

In [3]:
v_d = v_1*sin(delta - theta_1) 
v_q = v_1*cos(delta - theta_1) 

p_e = i_d*(v_d + R_a*i_d) + i_q*(v_q + R_a*i_q) 


ddelta = Omega_b*(omega - omega_s)
domega = 1/(2*H)*(p_m - p_e - D*(omega - omega_s))
de1q = 1/T1d0*(-e1q - (X_d - X1d)*i_d + v_f)
de1d = 1/T1q0*(-e1d + (X_q - X1q)*i_q)
        
g_1 = v_q + R_a*i_q + X1d*i_d - e1q
g_2 = v_d + R_a*i_d - X1q*i_q - e1d
g_3 = P_t - (v_1*v_0*sin(theta_1 - theta_0))/X_line
g_4 = Q_t + (v_1*v_0*cos(theta_1 - theta_0))/X_line - v_1**2/X_line
g_5 = i_d*v_d + i_q*v_q - P_t
g_6 = i_d*v_q - i_q*v_d - Q_t

h_1 = p_m
h_2 = p_e

sys = {'name':'smib_milano_ex8p1_4ord',
       'params_dict':params,
       'f_list':[ddelta,domega,de1q,de1d],
       'g_list':[g_1,g_2,g_3,g_4,g_5,g_6],
       'x_list':x_list,
       'y_ini_list':y_ini_list,
       'y_run_list':y_run_list,
       'u_run_dict':u_run_dict,
       'u_ini_dict':u_ini_dict,
       'h_dict':{'p_m':p_m,'p_e':p_e}}

sys = db.system(sys)
db.sys2num(sys)
                

In [4]:
sys

{'name': 'smib_milano_ex8p1_4ord',
 'params_dict': {'X_d': 1.81,
  'X1d': 0.3,
  'T1d0': 8.0,
  'X_q': 1.76,
  'X1q': 0.65,
  'T1q0': 1.0,
  'R_a': 0.003,
  'X_line': 0.05,
  'H': 3.5,
  'D': 1.0,
  'Omega_b': 314.1592653589793,
  'omega_s': 1.0,
  'v_0': 1.0,
  'theta_0': 0.0},
 'f_list': [Omega_b*(omega - omega_s),
  (-D*(omega - omega_s) - i_d*(R_a*i_d + v_1*sin(delta - theta_1)) - i_q*(R_a*i_q + v_1*cos(delta - theta_1)) + p_m)/(2*H),
  (-e1q - i_d*(-X1d + X_d) + v_f)/T1d0,
  (-e1d + i_q*(-X1q + X_q))/T1q0],
 'g_list': [R_a*i_q + X1d*i_d - e1q + v_1*cos(delta - theta_1),
  R_a*i_d - X1q*i_q - e1d + v_1*sin(delta - theta_1),
  P_t + v_0*v_1*sin(theta_0 - theta_1)/X_line,
  Q_t + v_0*v_1*cos(theta_0 - theta_1)/X_line - v_1**2/X_line,
  -P_t + i_d*v_1*sin(delta - theta_1) + i_q*v_1*cos(delta - theta_1),
  -Q_t + i_d*v_1*cos(delta - theta_1) - i_q*v_1*sin(delta - theta_1)],
 'x_list': ['delta', 'omega', 'e1q', 'e1d'],
 'y_ini_list': ['i_d', 'i_q', 'v_1', 'theta_1', 'p_m', 'v_f'],
 'y_r

In [5]:
for param in params:
    print(f'{param} = {params[param]};')
    

X_d = 1.81;
X1d = 0.3;
T1d0 = 8.0;
X_q = 1.76;
X1q = 0.65;
T1q0 = 1.0;
R_a = 0.003;
X_line = 0.05;
H = 3.5;
D = 1.0;
Omega_b = 314.1592653589793;
omega_s = 1.0;
v_0 = 1.0;
theta_0 = 0.0;


Matrix([
[                                                          0, 0, 0, 0],
[i_d*v_1*cos(delta - theta_1) - i_q*v_1*sin(delta - theta_1), 0, 0, 0]])